In [2]:
#pip install pyspark

In [10]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("sql operations").getOrCreate()

In [11]:
df=spark.read.json("/content/nyt2.json")

In [12]:
df.show()

+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|        price|   published_date|           publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {NULL, 27}|{{1212883200000}}|              Bantam| {1}|           {0}|           ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, NULL}|{{1212883200000}}|       Little, Br

In [13]:
df.dtypes

[('_id', 'struct<$oid:string>'),
 ('amazon_product_url', 'string'),
 ('author', 'string'),
 ('bestsellers_date', 'struct<$date:struct<$numberLong:string>>'),
 ('description', 'string'),
 ('price', 'struct<$numberDouble:string,$numberInt:string>'),
 ('published_date', 'struct<$date:struct<$numberLong:string>>'),
 ('publisher', 'string'),
 ('rank', 'struct<$numberInt:string>'),
 ('rank_last_week', 'struct<$numberInt:string>'),
 ('title', 'string'),
 ('weeks_on_list', 'struct<$numberInt:string>')]

In [14]:
df.count()

10195

In [15]:
df.head()

Row(_id=Row($oid='5b4aa4ead3089013507db18b'), amazon_product_url='http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author='Dean R Koontz', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', price=Row($numberDouble=None, $numberInt='27'), published_date=Row($date=Row($numberLong='1212883200000')), publisher='Bantam', rank=Row($numberInt='1'), rank_last_week=Row($numberInt='0'), title='ODD HOURS', weeks_on_list=Row($numberInt='1'))

In [16]:
df.columns

['_id',
 'amazon_product_url',
 'author',
 'bestsellers_date',
 'description',
 'price',
 'published_date',
 'publisher',
 'rank',
 'rank_last_week',
 'title',
 'weeks_on_list']

In [17]:
df.describe().show()

+-------+--------------------+---------------+--------------------+---------+------------------+
|summary|  amazon_product_url|         author|         description|publisher|             title|
+-------+--------------------+---------------+--------------------+---------+------------------+
|  count|               10195|          10195|               10195|    10195|             10195|
|   mean|                NULL|           NULL|                NULL|     NULL|1877.7142857142858|
| stddev|                NULL|           NULL|                NULL|     NULL| 370.9760613506458|
|    min|http://www.amazon...|        AJ Finn|                    |      ACE|  10TH ANNIVERSARY|
|    max|https://www.amazo...|various authors|’Tis for the Rebe...|allantine|               ZOO|
+-------+--------------------+---------------+--------------------+---------+------------------+



In [18]:
df.distinct().count()

10195

In [20]:
df.explain(extended=True)

== Parsed Logical Plan ==
Relation [_id#102,amazon_product_url#103,author#104,bestsellers_date#105,description#106,price#107,published_date#108,publisher#109,rank#110,rank_last_week#111,title#112,weeks_on_list#113] json

== Analyzed Logical Plan ==
_id: struct<$oid:string>, amazon_product_url: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>
Relation [_id#102,amazon_product_url#103,author#104,bestsellers_date#105,description#106,price#107,published_date#108,publisher#109,rank#110,rank_last_week#111,title#112,weeks_on_list#113] json

== Optimized Logical Plan ==
Relation [_id#102,amazon_product_url#103,author#104,bestsellers_date#105,description#106,price#107,published_dat

In [23]:
### Top 10 author names
df.select("author").show(10)

+--------------------+
|              author|
+--------------------+
|       Dean R Koontz|
|     Stephenie Meyer|
|        Emily Giffin|
|   Patricia Cornwell|
|     Chuck Palahniuk|
|James Patterson a...|
|       John Sandford|
|       Jimmy Buffett|
|    Elizabeth George|
|      David Baldacci|
+--------------------+
only showing top 10 rows



In [24]:
df.select("author","title","rank","price").show(10)

+--------------------+--------------------+----+-------------+
|              author|               title|rank|        price|
+--------------------+--------------------+----+-------------+
|       Dean R Koontz|           ODD HOURS| {1}|   {NULL, 27}|
|     Stephenie Meyer|            THE HOST| {2}|{25.99, NULL}|
|        Emily Giffin|LOVE THE ONE YOU'...| {3}|{24.95, NULL}|
|   Patricia Cornwell|           THE FRONT| {4}|{22.95, NULL}|
|     Chuck Palahniuk|               SNUFF| {5}|{24.95, NULL}|
|James Patterson a...|SUNDAYS AT TIFFANY’S| {6}|{24.99, NULL}|
|       John Sandford|        PHANTOM PREY| {7}|{26.95, NULL}|
|       Jimmy Buffett|          SWINE NOT?| {8}|{21.99, NULL}|
|    Elizabeth George|     CARELESS IN RED| {9}|{27.95, NULL}|
|      David Baldacci|     THE WHOLE TRUTH|{10}|{26.99, NULL}|
+--------------------+--------------------+----+-------------+
only showing top 10 rows



In [26]:
from pyspark.sql.functions import *

In [28]:
### when
df.select("title",when(df.title!="ODD HOURS",1).otherwise(0)).show(10)

+--------------------+-----------------------------------------------------+
|               title|CASE WHEN (NOT (title = ODD HOURS)) THEN 1 ELSE 0 END|
+--------------------+-----------------------------------------------------+
|           ODD HOURS|                                                    0|
|            THE HOST|                                                    1|
|LOVE THE ONE YOU'...|                                                    1|
|           THE FRONT|                                                    1|
|               SNUFF|                                                    1|
|SUNDAYS AT TIFFANY’S|                                                    1|
|        PHANTOM PREY|                                                    1|
|          SWINE NOT?|                                                    1|
|     CARELESS IN RED|                                                    1|
|     THE WHOLE TRUTH|                                                    1|

In [29]:
## isin
df[df.author.isin("Dean R Koontz","Stephenie Meyer")].show()

+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+---------+-------------+
|                 _id|  amazon_product_url|         author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|    title|weeks_on_list|
+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+---------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|  Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {NULL, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, NULL}|{{1212883200000}}|Little, Brown| {2}|           {1}| THE HOST|          {3}|
|{5b4aa4ead3089013...|http://www.amazon...|Stephenie Me

In [31]:
## like
df.select("author","title",df.title.like("%THE%")).show()

+--------------------+--------------------+----------------+
|              author|               title|title LIKE %THE%|
+--------------------+--------------------+----------------+
|       Dean R Koontz|           ODD HOURS|           false|
|     Stephenie Meyer|            THE HOST|            true|
|        Emily Giffin|LOVE THE ONE YOU'...|            true|
|   Patricia Cornwell|           THE FRONT|            true|
|     Chuck Palahniuk|               SNUFF|           false|
|James Patterson a...|SUNDAYS AT TIFFANY’S|           false|
|       John Sandford|        PHANTOM PREY|           false|
|       Jimmy Buffett|          SWINE NOT?|           false|
|    Elizabeth George|     CARELESS IN RED|           false|
|      David Baldacci|     THE WHOLE TRUTH|            true|
|        Troy Denning|          INVINCIBLE|           false|
|          James Frey|BRIGHT SHINY MORNING|           false|
|         Garth Stein|THE ART OF RACING...|            true|
|     Debbie Macomber|  

In [32]:
### startswith and endswith
df.select("author","title",df.title.startswith("THE")).show()

+--------------------+--------------------+----------------------+
|              author|               title|startswith(title, THE)|
+--------------------+--------------------+----------------------+
|       Dean R Koontz|           ODD HOURS|                 false|
|     Stephenie Meyer|            THE HOST|                  true|
|        Emily Giffin|LOVE THE ONE YOU'...|                 false|
|   Patricia Cornwell|           THE FRONT|                  true|
|     Chuck Palahniuk|               SNUFF|                 false|
|James Patterson a...|SUNDAYS AT TIFFANY’S|                 false|
|       John Sandford|        PHANTOM PREY|                 false|
|       Jimmy Buffett|          SWINE NOT?|                 false|
|    Elizabeth George|     CARELESS IN RED|                 false|
|      David Baldacci|     THE WHOLE TRUTH|                  true|
|        Troy Denning|          INVINCIBLE|                 false|
|          James Frey|BRIGHT SHINY MORNING|                 fa

In [34]:
df.select("author","title",df.title.endswith("LAND")).show()

+--------------------+--------------------+---------------------+
|              author|               title|endswith(title, LAND)|
+--------------------+--------------------+---------------------+
|       Dean R Koontz|           ODD HOURS|                false|
|     Stephenie Meyer|            THE HOST|                false|
|        Emily Giffin|LOVE THE ONE YOU'...|                false|
|   Patricia Cornwell|           THE FRONT|                false|
|     Chuck Palahniuk|               SNUFF|                false|
|James Patterson a...|SUNDAYS AT TIFFANY’S|                false|
|       John Sandford|        PHANTOM PREY|                false|
|       Jimmy Buffett|          SWINE NOT?|                false|
|    Elizabeth George|     CARELESS IN RED|                false|
|      David Baldacci|     THE WHOLE TRUTH|                false|
|        Troy Denning|          INVINCIBLE|                false|
|          James Frey|BRIGHT SHINY MORNING|                false|
|         

In [35]:
## update
df=df.withColumnRenamed("amazon_product_url","URL")

In [36]:
df.show()

+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+--------------------+-------------+
|                 _id|                 URL|              author| bestsellers_date|         description|        price|   published_date|           publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {NULL, 27}|{{1212883200000}}|              Bantam| {1}|           {0}|           ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, NULL}|{{1212883200000}}|       Little, Br

In [37]:
### Drop columns
df1=df.drop("bestsellers_date").show(5)

+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+
|                 _id|                 URL|           author|         description|        price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|Odd Thomas, who c...|   {NULL, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|           ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|Aliens have taken...|{25.99, NULL}|{{1212883200000}}|Little, Brown| {2}|           {1}|            THE HOST|          {3}|
|{5b4aa4ead3089013...|http://www.amazon...|     Emily Giffin|A woman's happy m..

In [38]:
## groupby
df.groupby("author").count().show()

+--------------------+-----+
|              author|count|
+--------------------+-----+
|          James Frey|    2|
|    Elin Hilderbrand|   58|
|   Sharon Kay Penman|    2|
|         Lisa Genova|    7|
|        Will Allison|    1|
|   Patricia Cornwell|   64|
|       Laurie R King|    6|
|          Tea Obreht|    6|
|        Sarah Dunant|    1|
|        Tim Johnston|    1|
|          Sara Gruen|   13|
|Tom Clancy with P...|    9|
|         Andre Dubus|    1|
|        Terry Brooks|   19|
|        Daniel Silva|   69|
|Karen White, Beat...|    1|
|      Jackie Collins|   14|
|          Pat Conroy|   11|
|          Ann B Ross|    5|
|      Michael Savage|    3|
+--------------------+-----+
only showing top 20 rows



In [41]:
### filter
df.filter(df["title"]=="SNUFF").show()

+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+-----+-------------+
|                 _id|                 URL|         author| bestsellers_date|         description|        price|   published_date|           publisher|rank|rank_last_week|title|weeks_on_list|
+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+--------------------+----+--------------+-----+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|Chuck Palahniuk|{{1211587200000}}|An aging porn que...|{24.95, NULL}|{{1212883200000}}|           Doubleday| {5}|           {0}|SNUFF|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|Chuck Palahniuk|{{1212192000000}}|An aging porn que...|{24.95, NULL}|{{1213488000000}}|           Doubleday|{10}|           {5}|SNUFF|          {2}|
|{5b4aa4ead3089013...|http://www.amazon.

In [42]:
df.select("author","title","description","price").write.save("Authors.json",format="json")

In [ ]:
### view this file
### save as parquet format